## 기본설정 및 함수정의

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
sejong = [36.51430420729354, 127.29034973889941 ]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 영등포구 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_002/_census_data_2023_bnd_dong_bnd_dong_29010_2023_2023"
sejong_gdf = gpd.read_file(shapefile_path)
sejong_gdf = sejong_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/4.세종시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

In [6]:
# 맵 객체 생성 및 데이터 로드
sejong_map = KeplerGl(height=1000, width=1500)
sejong_map.add_data(data=sejong_gdf, name="세종시 행정동 데이터")
sejong_map.add_data(data=grid_map_df, name="세종시 격자 데이터")
# 맵 출력 및 상세설정
sejong_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'세종시 행정동 데이터': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,…

In [40]:
# 맵 저장
sejong_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 기본정보 map.html")

Map saved to visualization/영등포구 기본정보 map.html!


## 영등포구 주민등록인구 현황

In [83]:
# 주민등록인구 data load
pop_df = pd.read_csv('SBJ_2406_001/2.영등포_주민등록인구현황.csv')
pop_df['std_ymd'] = pop_df['std_ymd'].astype(str)
pop_df['m_pop'] = pop_df['m_pop'].apply(lambda x : "".join(x.split(",")))
pop_df['fm_pop'] = pop_df['fm_pop'].apply(lambda x : "".join(x.split(",")))
pop_df['m_pop'] = pop_df['m_pop'].astype(int)
pop_df['fm_pop'] = pop_df['fm_pop'].astype(int)
pop_df['adm_nm'] = pop_df['adm_nm'].apply(lambda x:x.split(" ")[-1])
# 남성인구와 여성인구를 더하여 tot_pop(종합인구) column 생성
pop_df['tot_pop'] = pop_df.iloc[:,3:5].sum(axis=1)
pop_df = pop_df[pop_df['adm_nm'] != "영등포구"]
pop_df['adm_nm'] = pop_df['adm_nm'].str.replace('제', '')

In [ ]:
latest_pop = pop_df[pop_df['std_ymd'] == "202312"]
pop_dic = dict(zip(latest_pop['adm_nm'], latest_pop['tot_pop']))
ydp_pop_gdf = ydp_gdf.copy()
ydp_pop_gdf['pop'] = ydp_pop_gdf['ADM_NM'].map(pop_dic)

# 맵 객체 생성 및 데이터 로드
ydp_pop_map = KeplerGl(height=1000, width=1500)
ydp_pop_map.add_data(data=ydp_pop_gdf, name="영등포구 행정동-인구 데이터")
ydp_pop_map.add_data(data=grid_map_df, name="영등포구 격자 데이터")
ydp_pop_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")
# 맵 출력 및 상세설정
ydp_pop_map

In [89]:
# 맵 저장
ydp_pop_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 행정동별 인구 map.html")

Map saved to visualization/영등포구 행정동별 인구 map.html!


In [66]:
# target값 list 생성 및 평균값 높은 target순으로 정렬
targets = []
for target in pop_df['adm_nm'].unique():
    pop_hjd = pop_df[pop_df['adm_nm'] == (target)]
    targets.append([target,pop_hjd['tot_pop'].mean()])
targets = sorted(targets, key=lambda x : x[1], reverse=True)
targets = [target[0] for target in targets]

# 행정동별 시계열 인구추이 df정보를 mapping한 dict생성
hjd_pop_dic = {}
for target in targets:
    pop_hjd = pop_df[pop_df['adm_nm'] == (target)]
    pop_hjd = pop_hjd[['std_ymd', 'm_pop', 'fm_pop', 'tot_pop']].reset_index(drop=True)
    pop_hjd.rename(columns = {'m_pop' : '%s_남'%target, 'fm_pop' : '%s_여'%target, 'tot_pop' : '%s_총합'%target}, inplace = True)
    pop_hjd = pop_hjd.sort_values(by='std_ymd')
    hjd_pop_dic[target] = pop_hjd

In [ ]:
# 빈 그래프
fig = go.Figure()

# for문을 통해 라인을 추가
for target in targets:
    fig.add_trace(go.Scatter(x=hjd_pop_dic[target]['std_ymd'], y=hjd_pop_dic[target]['%s_총합'%target],
                            mode='lines+markers',
                            name=target))
# 색상 업데이트
colors =px.colors.sequential.Plasma # 컬러맵
for i, trace in enumerate(fig.data):
    trace.line.color = colors[i % len(colors)]
    
# 그래프 타이틀 및 레이블 설정
fig.update_layout(
    title='<b>행정동별 시계열 인구변화</b>',
    xaxis_title='연도',
    yaxis_title='인구수',
    width=1000,
    height=600
)

# 그래프 표시
fig.show()

In [ ]:
# Subplot figure(4x5)
fig = make_subplots(
    rows=3, 
    cols=6, 
    subplot_titles=["<b>%s</b>" % target for target in targets],
    vertical_spacing=0.05,  # 세로 여백
    horizontal_spacing=0.05  # 가로 여백
)

# 17개의 타겟 값에 대해 반복하여 그래프 추가
for i in range(17):
    df = hjd_pop_dic[targets[i]]
    # 각 subplot에 trace 추가
    row = (i // 6) + 1
    col = (i % 6) + 1
    fig.add_trace(go.Scatter(
        x=df['std_ymd'], 
        y=df['%s_총합'%targets[i]], 
        mode='lines',
        name='%s'%targets[i],
        line=dict(color='black')), 
        row=row, col=col, 
    )
    fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        tickvals=['202106', '202206', '202306'],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
    fig.update_yaxes(
        showticklabels=True, 
        showgrid=True,
    )
# 레이아웃 업데이트
fig.update_layout(height=1000, width=2200)

# 그래프 출력
fig.show()

### 영등포구 거주인구 필터링

국제연합(UN)의 기준에 따르면 전체 인구에서 65세 이상이 차지하는 비율인<br/>고령자 인구 비율이 7% 이상이면 고령화 사회, 14% 이상이면 고령 사회, 20% 이상이면 초고령 사회로 구분된다.

국제 연합 기준(UN)과 국내 국민연금 수급연령(연금 지급개시연령)<br/>및 근로자의 법상 정년(노동관계법령상 정년(60세) 의무규정)을 고려하여<br/>60대를 기준으로 고령화 비율을 나누었다. 

In [153]:
respop_df = pd.read_csv('SBJ_2406_001/1.영등포_성연령별_거주인구(격자).csv')

#grid ID 리스트 와 매핑 & respop_df 필터링
grid_id = grid_map_df['properties.gid'].tolist() # gid는 전부 unique한 값
grid_respop = respop_df[respop_df['gid'].isin(grid_id)]
grid_respop = grid_respop.fillna(0)
grid_respop = pd.concat([grid_respop.iloc[:, :2], grid_respop.iloc[:, 2:].astype(int)], axis=1)
grid_respop['year'] = grid_respop['year'].astype(str)

# 성별구분 없이 통합 및 세대별 인구수 데이터프레임으로 변환
columns = ['gid', 'year']
for i in range(2, len(grid_respop.columns)):
    age = f'{grid_respop.columns[i][2:4]}대' if grid_respop.columns[i][2] != '1' else f'{grid_respop.columns[i][2:5]}대'
    sex = '남' if grid_respop.columns[i][0] != 'm' else '여'
    columns.append('%s_%s'%(age, sex))
grid_respop.columns = columns
# 인구 column & 고령인구 column & 고령인구 비율(%) column 생성
grid_respop['인구'] = grid_respop.iloc[:, 2:].sum(axis=1)
grid_respop['고령인구'] = grid_respop.iloc[:, 10:-1].sum(axis=1)
grid_respop['고령인구_비율'] = grid_respop.iloc[:, -1] / grid_respop.iloc[:, -2]
grid_respop = grid_respop.fillna(0) # NaN값 0으로 대체
grid_respop['고령인구_비율'] = grid_respop['고령인구_비율'].apply(lambda x : round(x*100, 2))

#### 거주인구 연령대별 분포

In [ ]:
# 연령대별로 성별 통합
grid_respop['20대'] = grid_respop['20대_여'] + grid_respop['20대_남']
grid_respop['30대'] = grid_respop['30대_여'] + grid_respop['30대_남']
grid_respop['40대'] = grid_respop['40대_여'] + grid_respop['40대_남']
grid_respop['50대'] = grid_respop['50대_여'] + grid_respop['50대_남']
grid_respop['60대'] = grid_respop['60대_여'] + grid_respop['60대_남']
grid_respop['70대'] = grid_respop['70대_여'] + grid_respop['70대_남']
grid_respop['80대'] = grid_respop['80대_여'] + grid_respop['80대_남']
grid_respop['90대'] = grid_respop['90대_여'] + grid_respop['90대_남']
grid_respop['100대'] = grid_respop['100대_여'] + grid_respop['100대_남']

# 필요한 열만 선택
age_totals = grid_respop[['year', '20대', '30대', '40대', '50대', '60대', '70대', '80대', '90대', '100대']]

# 데이터프레임을 긴 형식으로 변환
age_totals_melted = age_totals.melt(id_vars='year', var_name='Age Group', value_name='Count')

# 연도별 연령대 분포 계산
age_totals_yearly = age_totals_melted[['Age Group', 'Count']].groupby(['Age Group']).sum().reset_index()
# 100대 연령대 제거
age_totals_yearly = age_totals_yearly[age_totals_yearly['Age Group'] != '100대']

# 연령대 종합 평균내기
age_totals_yearly['Count'] = age_totals_yearly['Count'] / 3
age_totals_yearly['Count'] = age_totals_yearly['Count'].astype(int)
# 시각화
fig = px.bar(
    age_totals_yearly,
    x='Age Group',
    y='Count',
    color='Count',
    color_continuous_scale='Plasma_r',  # Plasma 색상 맵 사용
    labels={'year': 'Year', 'Count': 'Population Count', 'Age Group': 'Age Group'},
)
fig.update_layout(height=600, width=1200)
fig.show()


#### 거주인구 연도별 변화(line + marker graph)

In [155]:
year_lst = range(2021, 2024)
year_pop = [] # 인구수 종합
elder_pop = [] # 고령인구 종합
elder_rate = [] # 고령인구 비율
non_elder_pop = [] # 비고령인구 종합
non_elder_rate = [] # 비고령인구 비율
for year in year_lst:
    # 해당 year의 인구수 종합
    year_pop.append(grid_respop[grid_respop['year'] == str(year)]['인구'].sum())
    # 해당 year의 고령인구 종합
    elder_pop.append(grid_respop[grid_respop['year'] == str(year)]['고령인구'].sum())
    # 해당 year의 고령인구 비율
    rate = grid_respop[grid_respop['year'] == str(year)]['고령인구'].sum() / grid_respop[grid_respop['year'] == str(year)]['인구'].sum()
    elder_rate.append(round(rate * 100, 2))
    # 해당 year의 비고령인구 종합
    non_elder = grid_respop[grid_respop['year'] == str(year)]['인구'].sum() - grid_respop[grid_respop['year'] == str(year)]['고령인구'].sum()
    non_elder_pop.append(non_elder)
    # 해당 year의 고령인구 비율
    rate = non_elder / grid_respop[grid_respop['year'] == str(year)]['인구'].sum()
    non_elder_rate.append(round(rate * 100, 2))

pop_df = pd.DataFrame({'연도':year_lst, '종합인구':year_pop,'고령인구':elder_pop, '비고령인구':non_elder_pop, '고령인구 비율':elder_rate, '비고령인구 비율':non_elder_rate})
pop_df

,연도,종합인구,고령인구,비고령인구,고령인구 비율,비고령인구 비율
0,2021,315512,87011,228501,27.58,72.42
1,2022,311843,87713,224130,28.13,71.87
2,2023,309588,89833,219755,29.02,70.98


In [ ]:
fig = px.line(pop_df, x='연도', y='종합인구', labels={'x':'연도', 'y':'인구수'})
fig.update_traces(mode='lines+markers')

# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['종합인구'][i] + 400,
        text=f"{pop_df['종합인구'][i]}명",
        showarrow=False,
    )
fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        range=[2020.5, 2023.5],
        tickvals=[2021, 2022, 2023],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
fig.update_yaxes(
    showticklabels=True, 
    showgrid=True,
    )
fig.update_layout(height=800, width=1200)
fig.show()

In [ ]:
fig = px.line(pop_df, x='연도', y=['고령인구', '비고령인구'], labels={'x':'연도', 'y':'인구수'})
fig.update_traces(mode='lines+markers')

# 각 마커 위에 주석을 추가
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['고령인구'][i] + 7000,
        text=f"{pop_df['고령인구'][i]}명",
        showarrow=False,
    )

for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['비고령인구'][i] + 7000,
        text=f"{pop_df['비고령인구'][i]}명",
        showarrow=False,
    )

fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        range=[2020.5, 2023.5],
        tickvals=[2021, 2022, 2023],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
fig.update_yaxes(
    showticklabels=True, 
    showgrid=True,
    )
# 범례 위치 설정 및 범례 이름 변경
fig.update_layout(
    legend=dict(x=1, y=1),
    legend_title_text='구분',
    height=800, 
    width=1200
)
fig.show()

In [ ]:
fig = px.line(pop_df, x='연도', y=['고령인구 비율', '비고령인구 비율'], labels={'x':'연도', 'y':'인구비율(%)'})
fig.update_traces(mode='lines+markers')
# 각 마커 위에 주석을 추가
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['고령인구 비율'][i] + 1,
        text=f"{pop_df['고령인구 비율'][i]}%",
        showarrow=False,
    )
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['비고령인구 비율'][i] + 1,
        text=f"{pop_df['비고령인구 비율'][i]}%",
        showarrow=False,
    )
fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        range=[2020.5, 2023.5],
        tickvals=[2021, 2022, 2023],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
fig.update_yaxes(
    showticklabels=True, 
    showgrid=True,
    )
# 범례 위치 설정 및 범례 이름 변경
fig.update_layout(
    legend=dict(x=1, y=1),
    legend_title_text='구분',
    height=800, 
    width=1200
)
fig.show()

#### 거주인구 시각화(격자매핑)

In [156]:
# 격자 ID와 격자 폴리곤 각각을 매핑한 딕셔너리 생성
map_dic = {}
for _, row in grid_map_df.iterrows():
    map_dic[row['properties.gid']] = row['geometry']

# 생성된 map_dic에 filtered_respop의 gid를 매핑하여 'geometry' column 생성
geometry_lst = []
for _, row in grid_respop.iterrows():
    geometry_lst.append(map_dic[row['gid']])
grid_respop['geometry'] = geometry_lst
grid_respop = gpd.GeoDataFrame(grid_respop, geometry='geometry')

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_respop_map = KeplerGl(height=1000, width=1500)
ydp_respop_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_respop_map.add_data(data=grid_respop, name="영등포구 격자-인구 데이터")

# 맵 출력 및 상세설정
ydp_respop_map

In [160]:
# 맵 저장
ydp_respop_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 거주인구 map(격자매핑).html")

Map saved to visualization/영등포구 현황/영등포구 거주인구 map(격자매핑).html!


#### 세대별 유동인구

In [189]:
# 유동인구 df 로드
floating_population_df = pd.read_csv('SBJ_2406_001/27.영등포_성연령별_유동인구.csv')
floating_population_df['STD_YM'] = floating_population_df['STD_YM'].astype(str)
floating_population_df['pop'] = floating_population_df.iloc[:, 1:-3].sum(axis=1).copy()
floating_population_df['STD_YM'] = floating_population_df['STD_YM'].apply(lambda x:x[:4])

# 연령대별로 성별 통합
floating_population_df['10대'] = floating_population_df['w_10g_pop'] + floating_population_df['m_10g_pop']
floating_population_df['20대'] = floating_population_df['w_20g_pop'] + floating_population_df['m_20g_pop']
floating_population_df['30대'] = floating_population_df['w_30g_pop'] + floating_population_df['m_30g_pop']
floating_population_df['40대'] = floating_population_df['w_40g_pop'] + floating_population_df['m_40g_pop']
floating_population_df['50대'] = floating_population_df['w_50g_pop'] + floating_population_df['m_50g_pop']
floating_population_df['60대'] = floating_population_df['w_60g_pop'] + floating_population_df['m_60g_pop']
col_lst = list(floating_population_df.columns[-6:]) + list(floating_population_df.columns[-7:-6]) + list(floating_population_df.columns[-9:-7])
floating_population_df = floating_population_df[col_lst]
floating_population_df = floating_population_df.groupby(['lon', 'lat']).mean().reset_index()
floating_population_df

,lon,lat,10대,20대,30대,40대,50대,60대,pop
0,126.878273,37.552921,0.323333,0.811111,1.671389,2.402778,2.266389,2.240556,9.070000
1,126.878276,37.552470,0.623056,1.660556,3.490278,4.966111,4.565000,4.723611,18.598889
2,126.878280,37.552020,0.636389,1.625556,3.372500,4.701667,4.421667,4.536111,18.021944
3,126.878284,37.551569,1.211944,2.606111,5.427222,7.280556,6.731667,7.047222,28.323333
4,126.878549,37.519573,2.923333,5.701667,12.430278,18.447500,19.311944,19.977222,72.642500
...,...,...,...,...,...,...,...,...,...
9517,126.949843,37.519476,0.405625,1.095312,1.339688,1.475625,1.505000,2.110313,7.067813
9518,126.949846,37.519025,0.252500,1.178333,1.551111,1.599167,1.438056,1.603333,7.030000
9519,126.949849,37.518574,0.123611,0.507778,0.873889,1.025000,0.940833,0.923611,4.113333
9520,126.949853,37.518124,0.112857,0.522000,0.662000,0.713143,0.655143,0.710000,3.111429


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_floatingpop_map = KeplerGl(height=1000, width=1500)
ydp_floatingpop_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_floatingpop_map.add_data(data=floating_population_df, name="영등포구 세대별 유동인구 데이터")
ydp_floatingpop_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_floatingpop_map

In [192]:
# 맵 저장
ydp_floatingpop_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 세대별 유동인구 map.html")

Map saved to visualization/영등포구 현황/영등포구 세대별 유동인구 map.html!


#### 시간대별 유동인구

In [ ]:
# 유동인구 df 로드
floating_population_df_T = pd.read_csv('SBJ_2406_001/28.영등포_시간대별_유동인구.csv')
floating_population_df_T['STD_YM'] = floating_population_df_T['STD_YM'].astype(str)
floating_population_df_T['STD_YM'] = floating_population_df_T['STD_YM'].apply(lambda x:x[:4])

# 열 이름 변경 과정
columns_to_rename = floating_population_df_T.columns[1:-2] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    numeric_part = column_name.split('_')[-1] # 숫자 부분 추출
    new_column_name = f"2020-01-01 {numeric_part}:00" # '시'를 붙여 새로운 열 이름 생성
    new_column_names.append(new_column_name) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_T.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_T =floating_population_df_T.iloc[:,1:]
floating_population_df_T = floating_population_df_T.groupby(['lon', 'lat']).mean().reset_index()

# 시간대별 유동인구 정보를 'timestamp' 형식으로 변환
def melt_time_columns(df):
    id_vars = ['lon', 'lat']
    value_vars = [col for col in df.columns if col not in id_vars]
    df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='time', value_name='population')
    return df_melted

floating_population_df_T = melt_time_columns(floating_population_df_T)
floating_population_df_T

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_floatingpop_T_map = KeplerGl(height=1000, width=1500)
ydp_floatingpop_T_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")
ydp_floatingpop_T_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_floatingpop_T_map.add_data(data=floating_population_df_T, name="영등포구 시간대별 유동인구 데이터")

# 맵 출력 및 상세설정
ydp_floatingpop_T_map

In [219]:
# 맵 저장
ydp_floatingpop_T_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 시간대별 유동인구 map.html")

Map saved to visualization/영등포구 현황/영등포구 시간대별 유동인구 map.html!


#### 요일별 유동인구

In [14]:
floating_population_df_D = pd.read_csv('SBJ_2406_001/29.영등포_요일별_유동인구.csv')

floating_population_df_D['STD_YM'] = floating_population_df_D['STD_YM'].astype(str)
floating_population_df_D['STD_YM'] = floating_population_df_D['STD_YM'].apply(lambda x:x[:4])

# 열 이름 변경 과정
columns_to_rename = floating_population_df_D.columns[1:-2] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    day_part = column_name.split('_')[0] # 숫자 부분 추출
    new_column_names.append(day_part) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_D.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_D =floating_population_df_D.iloc[:,1:]
floating_population_df_D = floating_population_df_D.groupby(['lon', 'lat']).mean().reset_index()

# 평일 및 주말 평균 계산
def calculate_weekday_weekend_means(df):
    # 평일 평균 = 1
    df['1'] = df[['mon', 'tue', 'wed', 'thu', 'fri']].mean(axis=1)
    # 주말 평균 = 2
    df['2'] = df[['sat', 'sun']].mean(axis=1)
    return df[['lon', 'lat', '1', '2']]

floating_population_df_D = calculate_weekday_weekend_means(floating_population_df_D)

# long 형식으로 변환
floating_population_df_D = floating_population_df_D.melt(
    id_vars=['lon', 'lat'],
    value_vars=['1', '2'],
    var_name='day_type',
    value_name='population'
)

floating_population_df_D

,lon,lat,day_type,population
0,126.878273,37.552921,1,12.285167
1,126.878276,37.552470,1,25.560667
2,126.878280,37.552020,1,24.682944
3,126.878284,37.551569,1,34.131389
4,126.878549,37.519573,1,84.020222
...,...,...,...,...
19237,126.949843,37.519476,2,7.803971
19238,126.949846,37.519025,2,6.366528
19239,126.949849,37.518574,2,3.624583
19240,126.949853,37.518124,2,2.748000


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_floatingpop_D_map = KeplerGl(height=1000, width=1500)
ydp_floatingpop_D_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")
ydp_floatingpop_D_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_floatingpop_D_map.add_data(data=floating_population_df_D, name="영등포구 평일 및 주말 유동인구 데이터")

# 맵 출력 및 상세설정
ydp_floatingpop_D_map

In [16]:
# 맵 저장
ydp_floatingpop_D_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 주중 및 주말 유동인구 map.html")

Map saved to visualization/영등포구 현황/영등포구 주중 및 주말 유동인구 map.html!
